### Supplementary Figure 6 | Treatment status modulates pathway-level responses to vaccination in sarcoidosis
Stratified pathway analyses compare transcriptional responses between treated and untreated sarcoidosis participants.

In [ ]:
## ============================================================
## SARC-only dream DiD contrasts: Sx+Tx vs Sx-NoTx
## Contrasts:
##   - Dose1: V1D7 − V1D0, Δ(Sx+Tx − Sx-NoTx)
##   - Dose2: V2D7 − V2D0, Δ(Sx+Tx − Sx-NoTx)
##   - Dose12: V2D7 − V1D0, Δ(Sx+Tx − Sx-NoTx)
##
## Outputs:
##   - Per contrast: gene CSV + fit RDS + Hallmark FGSEA CSV/RDS
##   - Bubble plot across contrasts (Hallmark only)
## ============================================================

suppressPackageStartupMessages({
  library(SummarizedExperiment)
  library(edgeR)
  library(limma)
  library(variancePartition)   # dream
  library(dplyr)
  library(tidyr)
  library(readr)
  library(stringr)
  library(forcats)
  library(ggplot2)
  library(msigdbr)
  library(fgsea)
  library(AnnotationDbi)
  library(org.Hs.eg.db)
  library(tibble)
  library(purrr)
})

## --------------------------
## 0) Setup and load paired cohort
## --------------------------
# Parallel setup
if (.Platform$OS.type == "windows") {
  BiocParallel::register(
    BiocParallel::SnowParam(workers = max(1, parallel::detectCores() - 1))
  )
} else {
  BiocParallel::register(
    BiocParallel::MulticoreParam(workers = max(1, parallel::detectCores() - 1))
  )
}
set.seed(42)

SCRIPT_REL_PATH <- NULL

if (is.null(SCRIPT_REL_PATH)) {
  # Interactive / notebook fallback: do not error, but warn if here() looks wrong
  if (!file.exists(here::here("README.md")) && !dir.exists(here::here(".git"))) {
    warning(
      "Repo root not detected via here(). ",
      "If paths are wrong, set your working directory to the repo root OR set SCRIPT_REL_PATH."
    )
  }
} else {
  here::i_am(SCRIPT_REL_PATH)
}

# ----------------------------
# Inputs
# ----------------------------
DATA_ROOT <- Sys.getenv("SARC_DATA_ROOT")

DATA_DIR <- here::here("data", "derived")
dir.create(DATA_DIR, recursive = TRUE, showWarnings = FALSE)

TAB_DIR <- here::here("tables", "main")
dir.create(TAB_DIR, recursive = TRUE, showWarnings = FALSE)

FIG_DIR <- here::here("figures", "main")
dir.create(FIG_DIR, recursive = TRUE, showWarnings = FALSE)

SUPP_FIG_DIR <- here::here("figures", "supplementary")
dir.create(SUPP_FIG_DIR, recursive = TRUE, showWarnings = FALSE)

if (DATA_ROOT == "") {
  # sensible default for your setup; change if needed
  DATA_ROOT <- "/mnt/z"
  Sys.setenv(SARC_DATA_ROOT = DATA_ROOT)
  message("SARC_DATA_ROOT was not set; using: ", DATA_ROOT)
}

# NOTE USES THE PAIRED ALL TIMEPOINTS SE FOR CONSISTENCY
if (!dir.exists(DATA_ROOT)) stop(glue("DATA_ROOT not found: {DATA_ROOT}"), call. = FALSE)
SE_RDS <- file.path(DATA_ROOT, "se_paired_all_timepoints.rds")
if (!file.exists(SE_RDS)) stop(glue("Missing input file: {SE_RDS}"), call. = FALSE)
se_all_full <- readRDS(SE_RDS)

stopifnot(exists("se_all_full"), is(se_all_full, "SummarizedExperiment"))

cd_all <- as.data.frame(colData(se_all_full))

## Metadata columns
visit_col <- intersect(c("time_point","m_time_point","visit","m_timepoint"), names(cd_all))[1]
group_col <- intersect(c("m_group","group","disease","m_disease_status"), names(cd_all))[1]   # SARC vs HC
id_col    <- intersect(c("m_subj2","participant","subject_id","subject","subj2"), names(cd_all))[1]
tx_col    <- intersect(c("m_txgroup","txgroup","tx_group","treatment_group"), names(cd_all))[1]
stopifnot(length(visit_col)==1, length(group_col)==1, length(id_col)==1, length(tx_col)==1)

message("Using columns: visit_col=", visit_col,
        "; disease/group_col=", group_col,
        "; id_col=", id_col,
        "; tx_col=", tx_col)

standardize_visit <- function(x) {
  x <- toupper(trimws(as.character(x)))
  x <- gsub("[ _-]+", "", x)
  x <- gsub("D00$", "D0", x)
  x <- gsub("D07$", "D7", x)
  x
}

standardize_group <- function(x) {
  x <- toupper(trimws(as.character(x)))
  dplyr::case_when(
    x %in% c("SARC","SARCOIDOSIS","SX") ~ "SARC",
    x %in% c("HC","HEALTHY","CONTROL","CONTROLS") ~ "HC",
    TRUE ~ x
  )
}

standardize_tx <- function(x) {
  x <- toupper(trimws(as.character(x)))
  # normalize a few common variants; adjust if your data use slightly different strings
  dplyr::case_when(
    x %in% c("SX-NOTX","SX_NOTX","SXNOTX","Sx-NoTx","SX-NOTX ") ~ "Sx-NoTx",
    x %in% c("SX+TX","SX_TX","SXTX","Sx+Tx")                   ~ "Sx+Tx",
    x == "HC"                                                  ~ "HC",
    TRUE                                                       ~ x
  )
}

## --------------------------
## 1) Helpers
## --------------------------
ens2sym <- function(ids) {
  ids2 <- sub("\\.\\d+$", "", as.character(ids))
  sym <- suppressMessages(
    AnnotationDbi::mapIds(
      org.Hs.eg.db,
      keys      = unique(ids2),
      keytype   = "ENSEMBL",
      column    = "SYMBOL",
      multiVals = "first"
    )[ids2]
  )
  sym <- unname(sym)
  ifelse(is.na(sym) | sym == "", ids2, sym)
}

clean_hallmark_labels <- function(x) {
  x |>
    stringr::str_replace("^HALLMARK_", "") |>
    stringr::str_replace_all("_", " ") |>
    stringr::str_to_title() |>
    stringr::str_replace_all("\\bTnfa\\b", "TNF-α") |>
    stringr::str_replace_all("\\bNf[- ]?Kb\\b", "NF-κB") |>
    stringr::str_replace_all("\\bBeta Catenin\\b", "β-Catenin") |>
    stringr::str_replace_all("\\bG2 M\\b", "G2/M") |>
    stringr::str_replace_all("\\bE2 F\\b", "E2F") |>
    stringr::str_replace_all("\\bIl6\\b", "IL6") |>
    stringr::str_replace_all("\\bJak\\b", "JAK") |>
    stringr::str_replace_all("\\bStat3\\b", "STAT3") |>
    stringr::str_replace_all("\\bPi3K\\b", "PI3K") |>
    stringr::str_replace_all("\\bAkt\\b", "AKT") |>
    stringr::str_replace_all("\\bMtor\\b", "mTOR") |>
    (\(s) stringr::str_replace_all(s, "\\bUv Response (Up|Dn)\\b", "UV response (\\1)"))()
}

get_msigdb_pathways <- function(category, subcategory = NULL, species = "Homo sapiens") {
  df <- msigdbr::msigdbr(species = species, category = category, subcategory = subcategory) |>
    dplyr::select(gs_name, gene_symbol)
  split(df$gene_symbol, df$gs_name)
}

run_fgsea_collection <- function(pathways, ranks, label_cleaner = clean_hallmark_labels,
                                 minSize = 15, maxSize = 500) {
  set.seed(1)
  fgsea::fgsea(pathways = pathways, stats = ranks, minSize = minSize, maxSize = maxSize) |>
    dplyr::arrange(padj) |>
    dplyr::mutate(
      direction = dplyr::if_else(NES > 0, "Δ(Sx+Tx) > Δ(Sx-NoTx)", "Δ(Sx+Tx) < Δ(Sx-NoTx)"),
      path_clean = label_cleaner(pathway)
    )
}

msig_H <- get_msigdb_pathways("H")

## --------------------------
## 2) Core: run one contrast in SARC-only
## --------------------------
run_did_contrast_sarc_tx <- function(visit_pair, contrast_tag, out_dir,
                                     fdr_filter_fgsea = 1.0) {
  dir.create(out_dir, showWarnings = FALSE, recursive = TRUE)

  cd <- as.data.frame(colData(se_all_full))
  md <- cd %>%
    dplyr::transmute(
      sample  = rownames(cd),
      subject = factor(trimws(as.character(.data[[id_col]]))),
      disease = standardize_group(.data[[group_col]]),
      visit   = standardize_visit(.data[[visit_col]]),
      tx_raw  = .data[[tx_col]],
      txgroup = standardize_tx(.data[[tx_col]])
    )

  # Keep SARC only, and only Sx-NoTx/Sx+Tx
  keep <- md$disease == "SARC" & md$txgroup %in% c("Sx-NoTx","Sx+Tx") & md$visit %in% visit_pair
  md   <- md[keep, , drop = FALSE]
  se   <- se_all_full[, md$sample, drop = FALSE]

  # Factors
  md$visit   <- factor(md$visit, levels = visit_pair)
  md$txgroup <- factor(md$txgroup, levels = c("Sx-NoTx","Sx+Tx"))  # reference = Sx-NoTx

  # Paired subjects: must have both visits in visit_pair
  paired_subj <- md %>%
    dplyr::group_by(subject) %>%
    dplyr::summarise(
      has_a = any(visit == visit_pair[1]),
      has_b = any(visit == visit_pair[2]),
      .groups = "drop"
    ) %>%
    dplyr::filter(has_a & has_b) %>%
    dplyr::pull(subject)

  md <- md %>% dplyr::filter(subject %in% paired_subj)
  se <- se[, md$sample, drop = FALSE]

  message(sprintf("[%s] SARC-only TX PAIRS — subjects: %d  (Sx-NoTx: %d, Sx+Tx: %d)",
                  contrast_tag,
                  dplyr::n_distinct(md$subject),
                  dplyr::n_distinct(md$subject[md$txgroup=="Sx-NoTx"]),
                  dplyr::n_distinct(md$subject[md$txgroup=="Sx+Tx"])))
  print(with(md, table(txgroup, visit)))

  # Counts -> edgeR -> voom
  cts <- assay(se, "counts"); if (!is.matrix(cts)) cts <- as.matrix(cts)
  storage.mode(cts) <- "integer"; cts[!is.finite(cts)] <- 0L
  stopifnot(identical(colnames(cts), md$sample))

  dge <- DGEList(cts)
  keep_genes <- rowSums(cpm(dge) >= 1) >= max(2, ceiling(0.2 * ncol(dge)))
  dge <- dge[keep_genes, , keep.lib.sizes = FALSE]
  dge <- calcNormFactors(dge)

  X <- model.matrix(~ txgroup * visit, data = md)
  v <- voom(dge, design = X, plot = FALSE)

  # dream mixed model
  form <- ~ txgroup * visit + (1|subject)
  fit <- dream(v, form, md)
  fit <- eBayes(fit)

  # Interaction coef: txgroupSx+Tx:visit<second level> (or swapped)
  coef_pattern <- paste0("txgroupSx\\+Tx:visit", visit_pair[2], "|visit", visit_pair[2], ":txgroupSx\\+Tx")
  coef_name <- grep(coef_pattern, colnames(coef(fit)), value = TRUE)
  stopifnot(length(coef_name) == 1)
  message("[", contrast_tag, "] Interaction coef: ", coef_name)

  # Gene table
  tt <- topTable(fit, coef = coef_name, number = Inf, sort.by = "P")
  tt$symbol  <- ens2sym(rownames(tt))
  tt$gene_id <- rownames(tt)
  tt <- tt %>% dplyr::select(symbol, gene_id, logFC, AveExpr, t, P.Value, adj.P.Val, B)

  write_csv(tt, file.path(TAB_DIR, paste0(contrast_tag, "_TX_DiD_gene_results_dream.csv")))
  saveRDS(list(fit=fit, md=md, coef_name=coef_name, form=form, visit_pair=visit_pair),
          file.path(DATA_DIR, paste0(contrast_tag, "_TX_dream_fit.rds")))

  # Ranks for FGSEA
  ranks <- tt$t
  names(ranks) <- tt$symbol
  ranks <- ranks[is.finite(ranks) & !is.na(names(ranks))]
  ranks <- tapply(ranks, names(ranks), function(x) x[which.max(abs(x))]) |> unlist()
  ranks <- sort(ranks, decreasing = TRUE)

  fg_H <- run_fgsea_collection(msig_H, ranks)

  # Optionally filter before saving (keep everything by default)
  fg_save <- fg_H
  if (is.finite(fdr_filter_fgsea)) fg_save <- fg_save %>% dplyr::filter(padj <= fdr_filter_fgsea)

  write_csv(fg_save, file.path(TAB_DIR, paste0(contrast_tag, "_TX_GSEA_Hallmark_dream.csv")))
  saveRDS(fg_H,      file.path(DATA_DIR, paste0(contrast_tag, "_TX_GSEA_Hallmark_dream.rds")))

  writeLines(capture.output(sessionInfo()), file.path(TAB_DIR, paste0(contrast_tag, "_sessionInfo.txt")))
  writeLines(paste("dream formula:", deparse(form)), file.path(TAB_DIR, paste0(contrast_tag, "_model.txt")))
  writeLines(paste("Interaction coefficient:", coef_name), file.path(TAB_DIR, paste0(contrast_tag, "_coef_name.txt")))

  message("[", contrast_tag, "] complete. Outputs in: ", TAB_DIR, " and ", DATA_DIR)
  invisible(fg_H)
}

## --------------------------
## 3) Run contrasts (SARC-only TX comparison)
## --------------------------
fg1 <- run_did_contrast_sarc_tx(c("V1D0","V1D7"), "Dose1",  out_dir      = DATA_DIR)
fg2 <- run_did_contrast_sarc_tx(c("V2D0","V2D7"), "Dose2",  out_dir      = DATA_DIR)
fg12<- run_did_contrast_sarc_tx(c("V1D0","V2D7"), "Dose12", out_dir      = DATA_DIR)

message("SARC-only TX contrasts complete.")

## ============================================================
## Hallmark bubble plot across TX contrasts
## ============================================================

# Contrast specs + correct file paths (match what run_did_contrast_sarc_tx wrote)
contrast_specs <- tibble::tribble(
  ~contrast_id, ~contrast_label,         ~file_H,
  "dose1",      "Dose 1 D7-D0",          file.path(TAB_DIR, "Dose1_TX_GSEA_Hallmark_dream.csv"),
  "dose2",      "Dose 2 D7-D0",          file.path(TAB_DIR, "Dose2_TX_GSEA_Hallmark_dream.csv"),
  "cum",        "Cumulative V2D7-V1D0",  file.path(TAB_DIR, "Dose12_TX_GSEA_Hallmark_dream.csv")
)

load_hallmark_csv <- function(path, label) {
  if (!file.exists(path)) {
    message("Skipping ", label, ": missing ", path)
    return(NULL)
  }
  readr::read_csv(path, show_col_types = FALSE) %>%
    dplyr::transmute(
      pathway_raw = path_clean,
      NES, padj,
      contrast = label
    )
}

# Use the actual column names: file_H and contrast_label
df_all <- purrr::map2_dfr(contrast_specs$file_H, contrast_specs$contrast_label, load_hallmark_csv)

if (nrow(df_all) == 0) {
  stop(
    "No Hallmark FGSEA CSVs found for TX contrasts.\nChecked:\n  ",
    paste(contrast_specs$file_H, collapse = "\n  "),
    call. = FALSE
  )
}

df_all$contrast <- factor(df_all$contrast, levels = contrast_specs$contrast_label)


## ---- Curated canonical mapping (reuse your mapping) ----
canon_map <- c(
  "interferon alpha response"         = "Interferon Alpha Response",
  "interferon gamma response"         = "Interferon Gamma Response",
  "inflammatory response"             = "Inflammatory Response",
  "tnfa signaling via nfkb"           = "TNF-α signaling via NF-κB",
  "il6 jak stat3 signaling"           = "IL6–JAK–STAT3 signaling",
  "complement"                        = "Complement",
  "coagulation"                       = "Coagulation",
  "apical junction"                   = "Apical Junction",
  "epithelial mesenchymal transition" = "Epithelial Mesenchymal Transition",
  "glycolysis"                        = "Glycolysis",
  "oxidative phosphorylation"         = "Oxidative Phosphorylation",
  "fatty acid metabolism"             = "Fatty Acid Metabolism",
  "adipogenesis"                      = "Adipogenesis",
  "xenobiotic metabolism"             = "Xenobiotic Metabolism",
  "reactive oxygen species pathway"   = "Reactive Oxygen Species Pathway",
  "unfolded protein response"         = "Unfolded Protein Response",
  "uv response up"                    = "UV Response (Up)",
  "hypoxia"                           = "Hypoxia",
  "g2m checkpoint"                    = "G2/M checkpoint",
  "mitotic spindle"                   = "Mitotic Spindle",
  "e2f targets"                       = "E2F Targets",
  "myc targets v1"                    = "Myc Targets V1",
  "myc targets v2"                    = "Myc Targets V2"
)

pathways_curated <- unname(canon_map)

category_map <- c(
  "Interferon Alpha Response"         = "Immune / cytokine",
  "Interferon Gamma Response"         = "Immune / cytokine",
  "Inflammatory Response"             = "Immune / cytokine",
  "TNF-α signaling via NF-κB"         = "Immune / cytokine",
  "IL6–JAK–STAT3 signaling"           = "Immune / cytokine",
  "Complement"                        = "Immune / cytokine",
  "Coagulation"                       = "Immune / cytokine",
  "Apical Junction"                   = "Immune / cytokine",
  "Epithelial Mesenchymal Transition" = "Immune / cytokine",
  "Glycolysis"                        = "Metabolic",
  "Oxidative Phosphorylation"         = "Metabolic",
  "Fatty Acid Metabolism"             = "Metabolic",
  "Adipogenesis"                      = "Metabolic",
  "Xenobiotic Metabolism"             = "Metabolic",
  "Reactive Oxygen Species Pathway"   = "Metabolic",
  "Unfolded Protein Response"         = "Cell cycle / stress",
  "UV Response (Up)"                  = "Cell cycle / stress",
  "Hypoxia"                           = "Cell cycle / stress",
  "G2/M checkpoint"                   = "Cell cycle / stress",
  "Mitotic Spindle"                   = "Cell cycle / stress",
  "E2F Targets"                       = "Cell cycle / stress",
  "Myc Targets V1"                    = "Cell cycle / stress",
  "Myc Targets V2"                    = "Cell cycle / stress"
)

df_clean <- df_all %>%
  dplyr::mutate(
    key = pathway_raw |>
      stringr::str_to_lower() |>
      stringr::str_replace_all("[^a-z0-9 ]", "") |>
      stringr::str_squish(),
    pathway = dplyr::recode(key, !!!canon_map, .default = pathway_raw)
  ) %>%
  dplyr::filter(pathway %in% pathways_curated) %>%
  dplyr::mutate(
    category = dplyr::recode(pathway, !!!category_map, .default = "Other"),
    direction = dplyr::if_else(NES >= 0, "Δ(SARC+Tx) > Δ(SARC-NoTx)", "Δ(SARC+Tx) < Δ(SARC-NoTx)"),
    neglog10FDR = -log10(padj),
    neglog10FDR = dplyr::if_else(is.infinite(neglog10FDR), NA_real_, neglog10FDR)
  ) %>%
  dplyr::filter(!is.na(neglog10FDR))

df_clean$neglog10FDR_capped <- pmin(df_clean$neglog10FDR, 16)

df_clean$category <- factor(
  df_clean$category,
  levels = c("Immune / cytokine", "Metabolic", "Cell cycle / stress")
)

# # Alphabetical top-to-bottom within category
# df_bubble <- df_clean %>%
#   dplyr::group_by(category) %>%
#   dplyr::arrange(category, pathway) %>%
#   dplyr::mutate(pathway = forcats::fct_rev(forcats::fct_inorder(pathway))) %>%
#   dplyr::ungroup()

# ---- Order pathways within each category by Dose 1 NES (ascending) ----
dose1_label <- "Dose 1 D7-D0"  # must match contrast_specs$contrast_label exactly

order_tbl <- df_clean %>%
  filter(contrast == dose1_label) %>%
  group_by(category, pathway) %>%
  summarise(order_key = mean(NES, na.rm = TRUE), .groups = "drop")  # mean is safe if duplicates exist

# Sanity check: did we actually find Dose 1 rows?
if (nrow(order_tbl) == 0) {
  stop("No rows found for contrast == '", dose1_label, "'. Check contrast labels.", call. = FALSE)
}

# Join the Dose 1 ordering key onto all contrasts
df_bubble <- df_clean %>%
  left_join(order_tbl, by = c("category", "pathway"))

# If any pathways are missing a Dose 1 NES (shouldn't happen if curated list is present),
# push them to the bottom within each category.
df_bubble <- df_bubble %>%
  mutate(order_key = if_else(is.na(order_key), Inf, order_key)) %>%
  group_by(category) %>%
  mutate(pathway = forcats::fct_reorder(pathway, order_key, .desc = TRUE)) %>%
  ungroup()


p_bubble <- ggplot(df_bubble, aes(x = NES, y = pathway)) +
  geom_point(aes(size = neglog10FDR_capped, fill = direction),
             shape = 21, colour = "black", alpha = 0.9) +
  facet_grid(
    rows = vars(category),
    cols = vars(contrast),
    scales = "free_y",
    space = "free_y",
    labeller = labeller(
    contrast = label_wrap_gen(width = 12)
    )
  ) +
  scale_fill_manual(
    name = "Direction",
    values = c("Δ(SARC+Tx) > Δ(SARC-NoTx)" = "#D55E00",
               "Δ(SARC+Tx) < Δ(SARC-NoTx)" = "#0072B2")
  ) +
  scale_size_continuous(
    name = "-log10(FDR)",
    range = c(2, 8),
    breaks = c(2, 4, 8, 12, 16),
    limits = c(0, 16)
  ) +
  labs(
    x = "NES (Δ(SARC+Tx) − Δ(SARC-NoTx))",
    y = NULL,
    title = "Hallmark pathway changes across vaccine doses (SARC only: Tx vs NoTx)",
    subtitle = "Bubble size = -log10(FDR), color = direction of Δ(SARC+Tx) − Δ(SARC-NoTx)"
  ) +
  theme_bw(base_size = 12) +
  theme(
    panel.grid.major.x = element_line(colour = "grey90"),
    panel.grid.major.y = element_blank(),
    panel.grid.minor   = element_blank(),
    strip.text.x       = element_text(face = "bold", size = 11),
    strip.text.y       = element_text(face = "bold", size = 8),
    axis.text.y        = element_text(size = 9),
    axis.text.x        = element_text(size = 10),
    legend.position    = "right",
    legend.title       = element_text(size = 10),
    legend.text        = element_text(size = 9),
    plot.title         = element_text(size = 14, face = "bold", margin = margin(b = 5)),
    plot.subtitle      = element_text(size = 11, margin = margin(b = 5))
  )

png_file <- file.path(SUPP_FIG_DIR, "Hallmark_bubble_TX_SARC_only_all_contrasts.png")
pdf_file <- file.path(SUPP_FIG_DIR, "Hallmark_bubble_TX_SARC_only_all_contrasts.pdf")

ggsave(png_file, p_bubble, width = 9, height = 8, dpi = 600)
ggsave(pdf_file, p_bubble, width = 9, height = 8, useDingbats = FALSE)

message("Saved TX bubble plot to:\n  ", png_file, "\n  ", pdf_file)

p_bubble


Using columns: visit_col=time_point; disease/group_col=m_group; id_col=m_subj2; tx_col=m_txgroup

[Dose1] SARC-only TX PAIRS — subjects: 6  (Sx-NoTx: 4, Sx+Tx: 2)



         visit
txgroup   V1D0 V1D7
  Sx-NoTx    4    4
  Sx+Tx      2    2


[Dose1] Interaction coef: txgroupSx+Tx:visitV1D7

[Dose1] complete. Outputs in: /mnt/z/sarcoidosis-mrna-vaccine-transcriptomics/tables/main and /mnt/z/sarcoidosis-mrna-vaccine-transcriptomics/data/derived

[Dose2] SARC-only TX PAIRS — subjects: 6  (Sx-NoTx: 4, Sx+Tx: 2)



         visit
txgroup   V2D0 V2D7
  Sx-NoTx    4    4
  Sx+Tx      2    2


[Dose2] Interaction coef: txgroupSx+Tx:visitV2D7

[Dose2] complete. Outputs in: /mnt/z/sarcoidosis-mrna-vaccine-transcriptomics/tables/main and /mnt/z/sarcoidosis-mrna-vaccine-transcriptomics/data/derived

[Dose12] SARC-only TX PAIRS — subjects: 6  (Sx-NoTx: 4, Sx+Tx: 2)

